In [57]:
import numpy as np

# Define the arrays for treasure and hunters
treasure = np.array([
            24, 70, 41, 21, 60, 
            47, 82, 87, 80, 35, 
            73, 89, 100, 90, 17, 
            77, 83, 85, 79, 55,
            12, 27, 52, 15, 30])

hunters = np.array([
            2, 4, 3, 2, 4, 
            3, 5, 5, 5, 3, 
            4, 5, 8, 7, 2, 
            5, 5, 5, 5, 4,
            2, 3, 4, 2, 3])

#treasure = np.array([24,70,47,82])
#hunters = np.array([2,4,3,5])

# Calculate the number of elements in the hunters array
n = len(hunters)

# Initialize the array y with zeros
y = np.zeros(n)

# Assign 100 to the last element of y
y[-1] = 100

# Loop to calculate differences as per the given formula
for i in range(n - 1):
    y[i] = hunters[i+1] / (treasure[i+1] * 7500) - hunters[i] / (treasure[i] * 7500)

# Print the resulting array y
#print(y)

A = np.zeros((n, n))

# Loop to populate the matrix for the diagonal and off-diagonal elements
for i in range(n - 1):
    A[i][i] = 1 / (treasure[i] * 7500)
    A[i][i+1] = -1 / (treasure[i+1] * 7500)

# Set the last row to all ones
A[n-1] = np.ones(n)

# Print the resulting matrix A
#print(A)

In [58]:
import numpy as np
from scipy.optimize import linprog

# Linear programming does not directly support equality constraints.
# Thus, we must separate A into two halves to express them as two inequality sets.
# We will minimize a zero function (as we're only interested in finding a feasible x).
c = np.zeros(n)  # Coefficients of the dummy objective function

# Bounds for each x_i: 0 < x_i < 1
x_bounds = [(0, 100) for _ in range(n)]

# Define constraints (Ax = y converted into Ax <= y and Ax >= y)
res = linprog(c, A_eq=A, b_eq=y, bounds=x_bounds, method='highs')

if res.success:
    print("Solution x:", res.x)
else:
    print("No solution found:", res.message)


No solution found: The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is At lower/fixed bound)


In [59]:
import numpy as np

# Try solving using least squares to see if a solution is fundamentally possible
x_ls, residuals, rank, s = np.linalg.lstsq(A, y, rcond=None)

# Check if the solution x_ls lies within the desired bounds
solution_within_bounds = np.all((x_ls > 0) & (x_ls < 100))

print("Least Squares Solution:", x_ls)
print("Solution within bounds:", solution_within_bounds)


Least Squares Solution: [ 1.35429769  5.78336827  2.73025856  0.93501048  4.38574423  3.56883298
  6.46051712  7.15932914  6.18099231  1.89168414  6.20265549  7.43885395
  5.97624039  5.57861635  0.37596087  5.7617051   6.60027952  6.87980433
  6.04122991  3.68693222 -0.32285115  0.77358491  3.267645    0.09643606
  1.19287212]
Solution within bounds: False


In [60]:
print(x_ls.reshape((5, 5)))

[[ 1.35429769  5.78336827  2.73025856  0.93501048  4.38574423]
 [ 3.56883298  6.46051712  7.15932914  6.18099231  1.89168414]
 [ 6.20265549  7.43885395  5.97624039  5.57861635  0.37596087]
 [ 5.7617051   6.60027952  6.87980433  6.04122991  3.68693222]
 [-0.32285115  0.77358491  3.267645    0.09643606  1.19287212]]


In [65]:
'''
Let us now mix this optimal distribution with other distributions
By the study from Jannik:
Naive: 19%
Optimists:3%
Spoilers: 1%
Rational: 62%
Mean (irrational): 15%

We will adapt these probabilities/playing styles to:
Spoilers: 5%
Naive: 20%
Optimists: 20%
Rational: 30%
Excel (mixed strategy off of the data): 20%
Meta-Rational: 5%
'''

'\nLet us now mix this optimal distribution with other distributions\nBy the study from Jannik:\nNaive: 19%\nOptimists:3%\nSpoilers: 1%\nRational: 62%\nMean (irrational): 15%\n\nWe will adapt these probabilities/playing styles to:\nSpoilers: 5%\nNaive: 25%\nOptimists: 5%\nRational:55%\nMeta-Rational:10%\n'

In [128]:
'''
Spoilers will just put all their guesses on the highest three tiles: 100, 90, and 89
'''
spoilers = 100*np.array([
            0, 0, 0, 0, 0, 
            0, 0, 0, 0, 0, 
            0, 1/3, 1/3, 1/3, 0, 
            0, 0, 0, 0, 0,
            0, 0, 0, 0, 0])

In [129]:
'''
Naive Players will uniformly pick between all numbers
'''

naive = 4*np.ones(n)

In [130]:
'''
Optimists will just assume everyone else plays naive,
and pick the tiles with highest payoff under these circumstances.
'''
payout = 7500*treasure/(hunters+4)
highest_indices = np.argsort(payout)[-12:]
print(highest_indices)
print(payout[highest_indices])

[13 12 15  1 18  8  6 10 16 17  7 11]
[61363.63636364 62500.         64166.66666667 65625.
 65833.33333333 66666.66666667 68333.33333333 68437.5
 69166.66666667 70833.33333333 72500.         74166.66666667]


In [131]:
'''
This leads us to define:
'''
optimists = np.zeros(n)

optimists[highest_indices[:2]] = 100*0.05
optimists[highest_indices[3:8]] = 100*0.08
optimists[highest_indices[8:]] = 100*0.1

In [132]:
'''
Rational players will assume everyone else is rational and hence they will follow the probabilities outlined:
'''
rational = x_ls

In [172]:
'''
Spreadsheet
'''
import numpy as np
from collections import Counter

# Your predefined array order
predefined_order = np.array([
    24, 70, 41, 21, 60, 
    47, 82, 87, 80, 35, 
    73, 89, 100, 90, 17, 
    77, 83, 85, 79, 55,
    12, 27, 52, 15, 30])

# List of numbers from your provided data
numbers = [
    73, 89, 100, 90, 89, 100, 90, 89, 73, 89, 100, 90, 89,
    87, 85, 47, 77, 60, 83, 55, 79, 82, 24, 35, 70, 41, 80,
    73, 80, 52, 85, 52, 83, 80, 90, 79, 89, 77, 52, 60, 70,
    73, 89, 85, 89, 70, 82, 87, 17, 21, 30, 30, 24, 17, 30, 
    17, 83, 100, 90, 85, 87, 47, 82, 82, 79, 85, 83, 60, 47, 
    89, 87, 77, 41, 83, 70, 73, 55, 52, 79, 77, 47, 77, 27, 
    80, 87, 70, 82, 70, 60, 79, 73, 80, 89, 87, 85, 100, 90, 50, 
    100, 89, 89, 79, 80
]

# Count the occurrences of each number
counts = Counter(numbers)

# Create a vector of zeros with the same length as predefined_order
excel = np.zeros_like(predefined_order, dtype=float)

# Map counts to the correct index in frequencies_vector based on predefined_order
for idx, value in enumerate(predefined_order):
    if value in counts:
        excel[idx] = counts[value] / len(numbers) * 100

print(excel.reshape(5,5))

[[ 1.94174757  5.82524272  1.94174757  0.97087379  3.88349515]
 [ 3.88349515  4.85436893  5.82524272  5.82524272  0.97087379]
 [ 5.82524272 11.65048544  5.82524272  5.82524272  2.91262136]
 [ 4.85436893  4.85436893  5.82524272  5.82524272  1.94174757]
 [ 0.          0.97087379  3.88349515  0.          2.91262136]]


In [153]:
'''
Meta Rational Players will do a similar calculation to us and find the optimum under this calculation.
'''
distr = (1/0.95)*(0.05*spoilers + 0.20*naive + 0.20*optimists + 0.3*rational + 0.20*excel)
meta_payout = 7500*treasure/(hunters+distr)
print(meta_payout[np.argsort(meta_payout)[-5:]])
print(np.argsort(meta_payout)[-5:])

[60139.86314692 60915.72585451 63878.21641605 64300.5256901
 66504.95508816]
[ 2  5  4 19 15]


In [176]:
meta_rational = 20*np.array([
            0, 0, 0, 0, 1, 
            1, 0, 0, 0, 0, 
            0, 0, 0, 0, 0, 
            1, 0, 0, 0, 1,
            0, 0, 1, 0, 0])

In [155]:
#Now calculate true payout
true_distr = 0.95*distr + 0.05*meta_rational
true_payout = 7500*treasure/(hunters+true_distr)
print(true_payout.round(1).reshape(5,5))

[[50074.6 56451.3 61408.8 48096.3 57016.7]
 [53028.7 59656.4 58715.8 57585.2 57544.6]
 [58085.  47582.  51994.6 50731.7 36477.4]
 [60793.4 57901.8 57802.2 57095.1 56550. ]
 [33294.6 47914.8 51607.8 39767.7 47464.5]]


In [136]:
print(true_distr.round(1).reshape(5,5))

[[1.4 6.1 1.8 1.3 2.3]
 [2.1 6.3 6.1 6.3 1.6]
 [5.3 9.6 7.6 7.5 1.1]
 [3.7 6.  6.1 6.2 2.1]
 [0.9 1.2 2.  1.  1.4]]


In [173]:
'''
Let us now simulate the different mixing distributions and average the payout.
'''
avg = np.zeros(n)

m = 500

alpha, beta = 1,5
x = np.random.beta(alpha, beta, size=m)

# Scale the results to sum to 30
x1 = x * 3/10
x2 = (1 - x) * 3/10


# Alpha parameters for the Dirichlet distribution
# Uniform distribution of alpha values will result in an equal expectation for the shares
alpha_params = [1.5, 2, 3.5]  # Equal concentration parameters

# Simulate using the Dirichlet distribution
samples = np.random.dirichlet(alpha_params, m)

# Scale the results by 10
scaled_samples = samples * 7/10

for i in range(m):
    distr = (1/0.97)*(x1[i]*spoilers + x2[i]*naive + scaled_samples[i][0]*optimists + scaled_samples[i][1]*rational + scaled_samples[i][2]*excel)
    meta_payout = 7500*treasure/(hunters+distr)
    z = np.zeros(n)
    z[np.argsort(meta_payout)[-5:]] = 20
    true = 0.97*distr + 0.03*z
    avg += 7500*treasure/(hunters+true)

avg /= m
print(avg.round(1).reshape(5,5))

[[45556.  55912.4 56508.5 44639.9 58976.1]
 [56298.4 57941.1 57867.5 57307.1 55599.8]
 [57793.1 45826.7 51759.8 50357.8 31222.9]
 [61360.5 57899.4 58165.7 56741.7 59112.4]
 [30672.9 45012.7 55566.4 37276.7 42792.9]]
